In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
print('Done')

Done


In [15]:
list_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source=requests.get(list_url).text
#print(source)
soup=BeautifulSoup(source,'xml')
table=soup.find('table')
column_names=['Postalcode','Borough','Neighborhood']
df=pd.DataFrame(columns=column_names)
#df.head()
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)]=row_data
df.head()

Postalcode           Borough               Neighborhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

In [16]:
df=df[df['Borough']!='Not assigned']
df.head()

Postalcode           Borough                                 Neighborhood
2        M3A        North York                                    Parkwoods
3        M4A        North York                             Victoria Village
4        M5A  Downtown Toronto                    Regent Park, Harbourfront
5        M6A        North York             Lawrence Manor, Lawrence Heights
6        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [17]:
df_temp=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_temp=df_temp.reset_index(drop=False)
df_temp.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
print('Done')

Done


In [18]:
df_merge=pd.merge(df,df_temp,on='Postalcode')
df_merge.drop(['Neighborhood'],axis=1,inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_merge.head()

Postalcode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [21]:
def get_geocode(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude
#print('Done')
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [39]:
df_geo.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merge = pd.merge(df_geo, df_merge, on='Postalcode')
#geo_merge.head()
geo_data=geo_merged[['Postalcode','Borough','Neighborhood',Latit]]

Postalcode   Latitude  Longitude      Borough  \
0        M1B  43.806686 -79.194353  Scarborough   
1        M1C  43.784535 -79.160497  Scarborough   
2        M1E  43.763573 -79.188711  Scarborough   
3        M1G  43.770992 -79.216917  Scarborough   
4        M1H  43.773136 -79.239476  Scarborough   

                             Neighborhood  
0                          Malvern, Rouge  
1  Rouge Hill, Port Union, Highland Creek  
2       Guildwood, Morningside, West Hill  
3                                  Woburn  
4                               Cedarbrae